In [ ]:
%%capture
%matplotlib inline
%load_ext iminizinc
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from ortools.constraint_solver import pywrapcp
plt.ion()

# Problem: n-Queens

In [ ]:
queens_slider = widgets.IntSlider(
    value=4,
    min=1,
    max=10,
    step=1,
    description='Queens:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(queens_slider)

### Example with Google OR-Tools

Model definition

In [ ]:
solver = pywrapcp.Solver("n-queens")
n = int(queens_slider.value) # size of board (n x n)
print("Queens: ", n)
# declare variables
q = [solver.IntVar(0, n - 1, "x%i" % i) for i in range(n)]

# constraints
solver.Add(solver.AllDifferent(q))
for i in range(n):
    for j in range(i):
        solver.Add(q[i] != q[j])
        solver.Add(q[i] + i != q[j] + j)
        solver.Add(q[i] - i != q[j] - j)

# solution and search
solution = solver.Assignment()
solution.Add([q[i] for i in range(n)])

## Collector to get all solutions
collector = solver.AllSolutionCollector(solution)

In [ ]:
# Solve
solver.Solve(
  solver.Phase([q[i] for i in range(n)], solver.INT_VAR_SIMPLE,
               solver.ASSIGN_MIN_VALUE), [collector])

num_solutions_plain = collector.SolutionCount()

In [ ]:
# Add a dumb symmetry breaking constraint
solver.Add(q[0] == 1)

# and solve it again
solver.Solve(
  solver.Phase([q[i] for i in range(n)], solver.INT_VAR_SIMPLE,
               solver.ASSIGN_MIN_VALUE), [collector])

num_solutions_symm = collector.SolutionCount()

**Problem summary**

The n-Queens problem with **{{ n }}** queens has
- {{ num_solutions_plain }} solutions if we allow symmetries
- {{ num_solutions_symm }} solutions if we don't allow symmetries

In [ ]:
### Example with MiniZinc

In [ ]:
%%minizinc --all-solutions

include "globals.mzn";
int: n;
array[1..n] of var 1..n: queens;
constraint all_different(queens);
constraint all_different([queens[i]+i | i in 1..n]);
constraint all_different([queens[i]-i | i in 1..n]);
solve :: int_search(queens, input_order, indomain_min, complete) satisfy;

In [ ]:
solutions = _
num_solutions_plain = len(solutions)

In [ ]:
%%minizinc --all-solutions

include "globals.mzn";
int: n;
array [1..n] of var 1..n: queens;

constraint all_different(queens);
constraint all_different([queens[i]+i | i in 1..n]);
constraint all_different([queens[i]-i | i in 1..n]);
constraint queens[1] == 2;

solve :: int_search(queens, input_order, indomain_min, complete) satisfy;

In [ ]:
solutions = _
num_solutions_symm = len(solutions)

The n-Queens problem with **{{ n }}** queens has
- {{ num_solutions_plain }} solutions if we allow symmetries
- {{ num_solutions_symm }} solutions if we don't allow symmetries